<a href="https://colab.research.google.com/github/Baah134/Baah134/blob/main/Whisper/Whisper_Transcription_3_ipyn_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai-whisper jiwer pandas

import os
import pandas as pd
import glob
import whisper
from jiwer import wer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

audio_folder = '/content/drive/MyDrive/DeepLearning/Whisper/my_data/audio'
csv_file = "/content/drive/MyDrive/DeepLearning/Whisper/my_data/transcriptions.csv"

Mounted at /content/drive


In [ ]:
from tqdm import tqdm

In [ ]:
df = pd.read_csv(csv_file)

# Dictionary {audio file name (without extension): transcription}
transcriptions = {os.path.splitext(row["filename"])[0]: row["transcription"] for _, row in df.iterrows()}


# Get all audio files (wav, m4a, etc.)
audio_files = glob.glob(os.path.join(audio_folder, "*"))


model = whisper.load_model("turbo")  # "small", "medium", "large" for better accuracy


matched_data = []  # Store results
total_wer = 0  # Sum of all WERs
valid_files = 0  # Count files with transcriptions

for audio_path in tqdm(audio_files, desc="Processing Audios"):
    file_name = os.path.basename(audio_path)  # Extract filename (e.g., "sample_001.wav")
    file_stem = os.path.splitext(file_name)[0]  # Remove extension (e.g., "sample_001")

    #
    ground_truth = transcriptions.get(file_stem)
    if ground_truth is None:
        continue  # Skip if no matching transcription

    # Transcribe audio using Whisper
    result = model.transcribe(audio_path)
    whisper_transcription = result["text"].strip()

    # Compute Word Error Rate (WER)
    error_rate = wer(ground_truth, whisper_transcription)

    # Store results
    matched_data.append((file_name, whisper_transcription, ground_truth, error_rate))
    total_wer += error_rate
    valid_files += 1

average_wer = total_wer / valid_files if valid_files else 0

# ============================
# STEP 6: SAVE RESULTS TO CSV
# ============================
output_csv = "whisper_results.csv"
df_results = pd.DataFrame(matched_data, columns=["Filename", "Whisper Transcription", "Ground Truth", "WER"])
df_results.to_csv(output_csv, index=False)

# Print Results
print(df_results)
print(f"\n✅ Average WER: {average_wer:.4f}")

100%|█████████████████████████████████████| 1.51G/1.51G [00:32<00:00, 50.5MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

          Filename                              Whisper Transcription  \
0   sample_029.wav  I need to finish this report before the deadline.   
1   sample_032.wav           De mizium ezibet showkiz egin artefakts.   
2   sample_033.wav  A heavy storm is expected to hit the coast ton...   
3   sample_007.wav  Tomorrow's forecast predicts heavy thunderstor...   
4   sample_006.wav  Innovation drives the future of artificial int...   
..             ...                                                ...   
56  sample_073.wav     They practice their lines for the school play.   
57  sample_074.wav     The machine learning model improved over time.   
58  sample_072.wav       The chef carefully plaited the gourmet dish.   
59  sample_063.wav            The alarm clock buzzed loudly at 6 a.m.   
60  sample_065.wav     A rainbow appeared outside the heavy rainfall.   

                                         Ground Truth       WER  
0   I need to finish this report before the deadline.  0.